# This is a PushShiftAPI exploration file. 

## In here, you can pull data from PushshiftAPI and save it as CSVs

In [1]:
from psaw import PushshiftAPI
import pandas as pd

api = PushshiftAPI()

## PushShiftAPI sanity test

This sanity tests scrapes the newest post on PushShiftAPI. Changes every time.

In [29]:
gen = api.search_submissions(filter=['url','author', 'title', 'subreddit'], limit=1)
for post in gen:
    print(post)
print("happened")

submission(author='WarnersBrosCA', created_utc=1644797487, subreddit='u_WarnersBrosCA', title='Is it a signal or a warning? Watch #TheBatman, In Theatres March 4.', url='https://ad.doubleclick.net/ddm/trackclk/N7217.131843.REDDIT.COM/B27153524.327036345;dc_trk_aid=519508530;dc_trk_cid=165617380;dc_lat=;dc_rdid={{ADVERTISING_ID}};tag_for_child_directed_treatment=;tfua=;ltd=', created=1644815487.0, d_={'author': 'WarnersBrosCA', 'created_utc': 1644797487, 'subreddit': 'u_WarnersBrosCA', 'title': 'Is it a signal or a warning? Watch #TheBatman, In Theatres March 4.', 'url': 'https://ad.doubleclick.net/ddm/trackclk/N7217.131843.REDDIT.COM/B27153524.327036345;dc_trk_aid=519508530;dc_trk_cid=165617380;dc_lat=;dc_rdid={{ADVERTISING_ID}};tag_for_child_directed_treatment=;tfua=;ltd=', 'created': 1644815487.0})
happened


/Users/albertlu/opt/anaconda3/envs/Antiwork/lib/python3.10/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


## Helper function that takes in a dataset & columns and saves as CSV

In [3]:
def save_as_csv(data, cols, label):
    df = pd.DataFrame(data, columns=cols)
    df.to_csv(label + ".csv")

# Main scraping function

## The function below scrapes all submission IDs & timestamps from r/antiwork from 2014-2021 and saves it as CSVs.

In [26]:
from datetime import datetime
from tqdm import tqdm
from pprint import pprint

def scrape_everything():
    all_ids = []
    params = ['id', 'created_utc']
    for year in range(2014, 2022):
        year = str(year)
        for month in range(1, 13):
            monthly_ids = []
            month = str(month)
            label = year + "-" + month
            if month == '12':
                start_epoch = int(datetime(int(year), int(month), 1).timestamp())
                end_epoch = int(datetime(int(year) + 1, 1, 1).timestamp())
            else:
                start_epoch = int(datetime(int(year), int(month), 1).timestamp())
                end_epoch = int(datetime(int(year), int(month) + 1, 1).timestamp())
            gen = api.search_submissions(after=start_epoch,
                                         before = end_epoch,
                                        subreddit='antiwork',
                                        filter=params,
                                        limit=None)
            for post in tqdm(gen):
                all_ids.append([post.id, post.created_utc])
                monthly_ids.append([post.id, post.created_utc])
            save_as_csv(monthly_ids, params, label)
            print("Got through " + year + "-" + month)
    save_as_csv(all_ids, params, "all_ids")

## This function merges two csvs

In [24]:
import os
# os.getcwd()
def merge_csv(csv1, csv2, name):
    a = pd.read_csv(csv1, index_col = 0)
    b = pd.read_csv(csv2, index_col = 0)

    merged = pd.concat([a, b], axis = 0, ignore_index = True)
    merged.to_csv(name + ".csv")